# Dowload and clean german Wikipedia dump

Dieses Notebook ist das Hauptnotebook, welches den Wikipedia Dump herunterlädt, diesen aufsplittet in exzellente und nicht exzellente artikel und anschließend eine grundlegende Datenaufbereitung durchführt. Die Aufgaben 1 & 2 sind für eine bessere Übersicht in den folgenden seperaten Notebooks bearbeitet worden:

Aufgabe 1: [Klassifizierung der Artikel](classification.ipynb)

Aufgabe 2: [Keyword extraktion](keywords.ipynb)

## Datenaufbereitung
Die Datenaufbereitung für beide Aufgaben wurde in diesem Notebook durchgeführt. Dieses Notebook muss __nicht__ ausgeführt werden, um die Evaluation durchzuführen. Dafür ist ein Subset generiert worden und kann genutzt werden. Ausschließlich das Training des Bert Klassifizierungsmodell benötigt die vollständigen Daten. Das ausführen dieses Notebooks dauert viele Stunden! 

### Install packages
Für Aplle Silicon Chips muss eine spezielle Version von pytorch verwendet werden. Daher werden die Installationen in den nachfolgenden Code-Blöcken unterschieden. Die Notebooks wurden mit einem Conda (Python 3.9) Kernel getestet und ausgeführt.

In [ ]:
# ONLY for apple slicon
! pip uninstall setuptools -y
! conda install pytorch torchvision torchaudio -c pytorch-nightly -y
! conda install numpy -y

In [ ]:
# ONLY for other cpus
! pip install numpy
! pip install pytorch

In [ ]:
! pip install -r requirements.txt

### Import Packages

In [2]:
# imports
import sys
import os
import bz2
import requests
import shutil
import csv
import pandas as pd

# regex
import re

# sklearn
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

# read wikipedia dump
import mwxml
# data cleaning
import html2text
import wikitextparser as wtp
# text metrics
import textstat

# multithreading
from threading import Thread


### Static Variables / Config

In [4]:
# static var
DUMP_URL = 'https://dumps.wikimedia.org/dewiki/latest/dewiki-latest-pages-articles.xml.bz2'
DUMP_FILE_ZIP = './dewiki-latest-pages-articles.xml.bz2'
DUMP_FILE_ENTPACKT = './dewiki-latest-pages-articles.xml'

EXZELLENT_FOLDER = './data/exzellent'
NOT_EXZELLENT_FOLDER = './data/not_exzellent'
SUBSET_FOLDER = './data/subset'

CSV_FILE = './articles_meta.csv'

### Download XML Dump herunterladen und Chunkweise abspeichern

Herunterladen des Wikipedia Dumps mit allen deutschsprachigen Artikeln von Wikimedia. Es wurde sich gegen die API entschieden, da hier die Artikel alle einzeln heruntergeladen werden müssen und somit die Verarbeitungszeit für die 2 Mio. Artikel deutlich höher wäre. Der Dump entspricht der aktuellsten verfügbaren Version und enthält alle Wikipedia Artikel im XML-Format. Da die große Datei nicht auf einmal im Arbeitsspeicher geladen werden kann, wird diese in Chunks unterteilt und abgespeichert. 

In [ ]:
# function for downloading the Wikipedia dump chunk for chunk to reduce ram usage
def download_file(url, file_path):
    response = requests.get(url, stream=True)
    with open(file_path, 'wb') as file:
        # write chunk in file
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)

# download wikipedia dump
download_file(DUMP_URL, DUMP_FILE_ZIP)

### XML Dump entpacken

Um den Wikipedia Dump nutzen zu können, muss dieser entpackt werden.

In [11]:
# unzip the xml-dump and save it
with open(DUMP_FILE_ENTPACKT, 'wb') as new_file, bz2.BZ2File(DUMP_FILE_ZIP, 'rb') as file:
    for data in iter(lambda : file.read(100 * 1024), b''):
        new_file.write(data)

### Artikel aufbereiten und sortieren nach Label

Für die bearbeitung der Aufgaben müssen die Artikel vorverarbeitet werden. Für eine einfachere Nutzung der Artikel werden diese aus der XML-Datei gelesen und in einzelnen Text Dateien gespeichert. Zudem wird der Text vorverarbeitet. Es werden diverse HTML und Markdown ähnliche Tags entfernt. Zudem werden bereits in der Aufbereitung einige features der Artikel berechnet bzw. gezählt und anschließend in einer CSV Datei abgelegt. Die einzelnen Artikel werden in eigenen Threads bearbeitet, um die Verarbeitungszeit zu verkürzen. Das zusammenführen der Features erfolgt wieder gesammelt im Main-Thread.

Ein Beispiel-Artikel vor der Aufbereitung kann hier betrachtet werden: [explanation/160.xml](./explanation/160.xml) 

Der selbe Artikel nach der Aufbereitung finden Sie hier: [explanation/160.txt](./explanation/160.txt)

In [8]:
class CleanSaveArticleThread(Thread):
    def __init__(self, *args):
        Thread.__init__(self)
        # get given args
        self.page = args[0]
        self.revision = args[1]

        # initalize vars
        self.number_images = 0
        self.number_citations = 0
        self.number_headers = 0
        self.number_links = 0
        self.number_categories = 0

        self.saved = False
        self.is_excellent = False

        # set language of textstat
        textstat.set_lang("de")

    # override the run function
    def run(self):
        # get text from revision
        text = self.revision.text

        # check if article is excellent
        PATTERN_EXCELLENT = r"\{\{Exzellent\|"
        x = re.search(PATTERN_EXCELLENT, text)
        if x is not None:
            self.is_excellent = True
        else:
            self.is_excellent= False

        # filter if article is only redirect and has no text 
        PATTERN_REDIRECT = r"(#REDIRECT|#redirect|#WEITERLEITUNG)"
        
        if re.search(PATTERN_REDIRECT, self.revision.text):
            # with open(os.path.join('./data/trash', str(page.id) + '.txt'), "x") as f:
            #     f.write(page.title + "\n" + text)
            self.saved = False
            return


        # feature extraction for classification task
        # count images in article
        PATTERN_IMAGES = r"\[\[Datei:[^\]]+\.(?:jpg|png|svg)[^\]]+\]\]"
        self.number_images = len(re.findall(PATTERN_IMAGES, self.revision.text))

        # count citations in article
        PATTERN_CITATIONS = r"\/ref"
        self.number_citations = len(re.findall(PATTERN_CITATIONS, self.revision.text))

        # count headers
        PATTERN_HEADER = r"==+ (.*?) ==+"
        self.number_headers = len(re.findall(PATTERN_HEADER, self.revision.text))

        # count link to other wikipedia articles
        PATTERN_LINK = r"\[\[(?!(?:.*\bDatei:\b.*|.*Kategorie:))([^]]+)\]\]"
        self.number_links = len(re.findall(PATTERN_LINK, self.revision.text))

        # count categories of the article
        PATTERN_CATEGORIE = r"\[\[Kategorie:[^\]]+\]\]"
        self.number_categories = len(re.findall(PATTERN_CATEGORIE, self.revision.text))


        # text cleanup
        # entnommen aus: https://github.com/daveshap/PlainTextWikipedia
        try:
            # Plain Text
            text = wtp.parse(text).plain_text()  
            # Remove HTML
            text = html2text.html2text(text)
        
            # Replace newlines
            text = text.replace('\\n', ' ')
            # Replace excess whitespace
            text = re.sub('\s+', ' ', text)
        except:
            self.saved = False
            return
        # end entnommen aus

        # calculate metrics / features for classification task
        # count number of words
        self.number_words = textstat.lexicon_count(text, removepunct=True)

        # count number of scentens
        self.number_scentens = textstat.sentence_count(text)

        try:
            # calculate Wiener Sachtextformel
            self.wiener_sachtextformel = textstat.wiener_sachtextformel(text, variant=1)
        except:
            self.saved = False
            return



        # save articles as txt file in correct folder
        if self.is_excellent:
            # filter excellent label from article (just to be sure is not in article anymore - usually the html2text function filtes these tags)
            text = text.replace('\{\{Exzellent|', '\{\{')
            # set target folder based on label
            target_folder = EXZELLENT_FOLDER
        else: 
            # set target folder based on label
            target_folder = NOT_EXZELLENT_FOLDER
        
        # save in target folder and add Wikipedia title in first line of document
        with open(os.path.join(target_folder, str(self.page.id) + '.txt'), "x") as f:
            f.write(self.page.title + "\n" + text)
            f.close()
            
        # set article is saved var
        self.saved = True

In [9]:
def write_meta_csv(thread: CleanSaveArticleThread) -> None:
    # check if article is saved
    if thread.saved:
        # write meta data to csv file
        with open(CSV_FILE, 'a') as csv_file:
            writer = csv.writer(csv_file)
            writer.writerow([
                thread.page.id, 
                thread.is_excellent, 
                thread.number_images, 
                thread.number_citations, 
                thread.number_headers, 
                thread.number_links, 
                thread.number_categories,
                thread.number_words,
                thread.number_scentens,
                thread.wiener_sachtextformel
                ])
            csv_file.close()

In [17]:
# create exzellent folder if not exists otherwise remove existing folder
print ("removing existing folders and files")
if os.path.exists(EXZELLENT_FOLDER):
    shutil.rmtree(EXZELLENT_FOLDER)
os.makedirs(EXZELLENT_FOLDER)

# create not exzellent folder if not exists otherwise remove existing folder
if os.path.exists(NOT_EXZELLENT_FOLDER):
    shutil.rmtree(NOT_EXZELLENT_FOLDER)
os.makedirs(NOT_EXZELLENT_FOLDER)

# create csv file for meta data
header = [
    'article_id',
    'is_excellent',
    'number_images',
    'number_citations',
    'number_headers',
    'number_links',
    'number_categories',
    'number_words',
    'number_scentens', 
    'wiener_sachtextformel'
    ]

# remove csv file if exists
if os.path.exists(CSV_FILE):
    os.remove(CSV_FILE)

# write header to meta csv file
with open(CSV_FILE, 'w+') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(header)


# define wikipedia dump
dump = mwxml.Dump.from_file(open(DUMP_FILE_ENTPACKT))

count_articles = 0
cleaned_saved = 0

thread_list = []

# print some information about the wikipedia dump
print("### Wikipedia Dump ###")
print(dump.site_info.name, dump.site_info.dbname)

print("### Read Articles ###")
# for schleifen entnommen aus: 
for idx_page, page in enumerate(dump):
    for idx_revision, revision in enumerate(page):
        if revision.text is not None:

            count_articles += 1

            # start basic cleaning of article in seperated Thread for better performance
            new_thread = CleanSaveArticleThread(page, revision)
            new_thread.start()
            thread_list.append(new_thread)
            
            # update output
            sys.stdout.write('\r -reading- Erfasste Artikel: %i, davon vorverarbeitet: %i   ' % (count_articles, cleaned_saved))
            sys.stdout.flush()


            # save all information in csv file
            if(len(thread_list) >= 500):
                sys.stdout.write('\r -add csv- Erfasste Artikel: %i, davon vorverarbeitet: %i   ' % (count_articles, cleaned_saved))
                sys.stdout.flush()

                for thread in thread_list:
                    # wait until thread ist done
                    thread.join()
                    
                    # write data of thread to meta csv file
                    write_meta_csv(thread)

                    # increase number of saved files
                    cleaned_saved += 1

                # remove all threads 
                thread_list = []

# save remaining article meta to csv
for thread in thread_list:
    # wait until thread ist done
    thread.join()
    
    # write data of thread to meta csv file
    write_meta_csv(thread)

    # increase number of saved files
    cleaned_saved += 1

print('\n Anzahl der erfassten Artikel: %i' % count_articles)

removing existing folders and files
### Wikipedia Dump ###
Wikipedia dewiki
### Read Articles ###
 -reading- Erfasste Artikel: 211689, davon vorverarbeitet: 211500   

IOStream.flush timed out


 -reading- Erfasste Artikel: 1922698, davon vorverarbeitet: 1922500   

IOStream.flush timed out


 -reading- Erfasste Artikel: 2091181, davon vorverarbeitet: 2091000   

### Subset generieren
Zum generieren eines Subsets wird dieselbe train, val, test aufteilung genutzt, wie auch bei der classification Aufgabe. Es wird das Testdatenset als Subset abgespeichert. Dieses Subset ist zudem in dem GitHub Repository enthalten. Die Ergebnisse können dadurch bei der Evaluierug auch ohne herunterladen, entzippen und Datenaufbereitung des Wikipedia Dumps reproduziert werden.

In [19]:
# generate subset equivalent to the test dataset 
print('remove existing subset')
if os.path.exists(SUBSET_FOLDER):
    shutil.rmtree(SUBSET_FOLDER)
print('generate subset folders')
os.makedirs(SUBSET_FOLDER)
os.makedirs(SUBSET_FOLDER+'/exzellent')
os.makedirs(SUBSET_FOLDER+'/not_exzellent')

# read filenams from meta csv
print('read meta data')
original_meta_data = pd.read_csv(CSV_FILE, header=0, index_col=0)
X = original_meta_data.drop(['is_excellent'], axis=1)
Y = original_meta_data['is_excellent']

# undersample data for same number of text per class 
print('undersample original dataset')
rus = RandomUnderSampler(random_state=42)
X, Y = rus.fit_resample(X, Y)

# gernerate train, val, test set
print('split dataset')
_, X_test_val, _, Y_test_val = train_test_split(X, Y, test_size=0.3, random_state=42)
X_test, _, Y_test, _ = train_test_split(X_test_val, Y_test_val, test_size=0.5, random_state=42)

# concat x and y to df
dataset = pd.concat([X_test,Y_test], axis=1)

# save articles in subset folders
print('save articles')
for article_id, article in dataset.iterrows():
    # get folder name based on label
    ordner_original = EXZELLENT_FOLDER if article['is_excellent'] else NOT_EXZELLENT_FOLDER
    ordner_subset = SUBSET_FOLDER + str('/exzellent' if article['is_excellent'] else '/not_exzellent')
    # merge filename
    filename = str(article_id) + '.txt'
    # join filepath
    filepath_original = os.path.join(ordner_original, filename)
    filepath_subset = os.path.join(ordner_subset, filename)
        
    # copy original file to subset folder
    shutil.copyfile(filepath_original, filepath_subset)

# write meta csv for the subset
print('write new meta csv')
dataset.to_csv(os.path.join(SUBSET_FOLDER,'articles_meta.csv'), mode='w+')

# print information about the subset
print('----')
print('Gesamtanzahl der Artikel: %s' % len(original_meta_data))
print('Anzahl der Samples im Subset (Testdaten): %i' % len(Y_test))
print('Klassenverteilung: %s' % Counter(Y_test))

remove existing subset
generate subset folders
read meta data
undersample original dataset
split dataset
save articles
write new meta csv
----
Gesamtanzahl der Artikel: 2496675
Anzahl der Samples im Subset (Testdaten): 807
Klassenverteilung: Counter({True: 411, False: 396})
